In [1]:
pip install pandas numpy tensorflow scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

In [3]:
# Đọc file CSV chứa thông tin sản phẩm
df = pd.read_csv('./new_image_product.csv')

In [4]:
# Gán nhãn cho các sản phẩm dựa trên productname
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['productname'])

In [5]:
# Lưu lại label encoder cho Flask API sau này
np.save('label_classes.npy', label_encoder.classes_)

In [6]:
# Chuẩn bị tập dữ liệu hình ảnh và nhãn
X = []
y = []

In [7]:
# Duyệt qua từng dòng trong DataFrame
for _, row in df.iterrows():
    img_path = os.path.join('', row['imagepath'])
    img = load_img(img_path, target_size=(224, 224))  # Chuyển đổi ảnh về kích thước 224x224
    img_array = img_to_array(img)
    X.append(img_array)
    y.append(row['label'])

In [8]:
# Chuyển danh sách thành mảng NumPy
X = np.array(X)
y = np.array(y)

In [9]:
# Chuẩn hóa hình ảnh về khoảng [0, 1]
X = X / 255.0

In [10]:
# Tách dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Tạo mô hình ResNet50 pre-trained
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [12]:
# Freeze các lớp của mô hình ResNet50
for layer in base_model.layers:
    layer.trainable = False

In [13]:
# Thêm các lớp fully connected
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Lớp đầu ra với số lượng nhãn
])

In [14]:
# Compile và huấn luyện mô hình
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.0085 - loss: 4.4772 - val_accuracy: 0.0000e+00 - val_loss: 4.4751
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.0085 - loss: 4.4088 - val_accuracy: 0.0000e+00 - val_loss: 4.4344
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.0148 - loss: 4.3344 - val_accuracy: 0.0000e+00 - val_loss: 4.4032
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.0148 - loss: 4.3046 - val_accuracy: 0.0000e+00 - val_loss: 4.3760
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.0148 - loss: 4.2744 - val_accuracy: 0.0000e+00 - val_loss: 4.3521
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.0085 - loss: 4.2766 - val_accuracy: 0.0000e+00 - val_loss: 4.3376
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0085 - loss: 4.2425 - val_accuracy: 0.0000e+00 - val_loss: 4.3253
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.0148 - loss: 4.1783 - val_accuracy: 0.0000

In [15]:
# Unfreeze một số lớp cuối của ResNet50 để fine-tune
for layer in base_model.layers[-10:]:
    layer.trainable = True

In [16]:
# Compile lại và tiếp tục huấn luyện
model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.0315 - loss: 4.1313 - val_accuracy: 0.0323 - val_loss: 4.3014
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0629 - loss: 4.0897 - val_accuracy: 0.0323 - val_loss: 4.2997
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0475 - loss: 4.0686 - val_accuracy: 0.0323 - val_loss: 4.2979
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.1031 - loss: 4.0070 - val_accuracy: 0.0323 - val_loss: 4.2964
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.0865 - loss: 3.9668 - val_accuracy: 0.0323 - val_loss: 4.2956
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0890 - loss: 3.9543 - val_accuracy: 0.0323 - val_loss: 4.2948
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.0981 - loss: 3.9051 - val_accuracy: 0.0323 - val_loss: 4.2945
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.0960 - loss: 3.8943 - val_accuracy: 0.0323 - val_loss: 4.2943
Epoch 9/

In [24]:
# Huấn luyện mô hình như trước đó
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.1210 - loss: 3.8099 - val_accuracy: 0.0323 - val_loss: 4.2902
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.1223 - loss: 3.8063 - val_accuracy: 0.0323 - val_loss: 4.2880
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.1531 - loss: 3.7610 - val_accuracy: 0.0323 - val_loss: 4.2856
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.1654 - loss: 3.6966 - val_accuracy: 0.0323 - val_loss: 4.2836
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.1748 - loss: 3.6797 - val_accuracy: 0.0323 - val_loss: 4.2816
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step - accuracy: 0.1554 - loss: 3.6288 - val_accuracy: 0.0323 - val_loss: 4.2800
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.2054 - loss: 3.5948 - val_accuracy: 0.0323 - val_loss: 4.2782
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 4s/step - accuracy: 0.1900 - loss: 3.6006 - val_accuracy: 0.0323 - val_loss: 4.2771
Epoch 9/